In [3]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader ,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

# Loading API keys
load_dotenv()

True

In [4]:
# extarct the data from PDF
def load_pdf(data_dir):
    loader =DirectoryLoader(data_dir,
                            glob = "*.pdf",
                            loader_cls =PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf(r"C:\github\openai\athina-ai\data")

In [6]:
# creating textn chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 50000,
        chunk_overlap = 0
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
docs = text_split(extracted_data)

In [8]:
docs =docs[1:24]

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
def download_hugging_face_embdinds():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embdinds()

c:\github\openai\athina-ai\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\github\openai\athina-ai\env\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [1]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

NameError: name 'os' is not defined

In [13]:
from langchain_openai import ChatOpenAI
# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")

In [16]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)
testset = generator.generate_with_langchain_docs(docs, test_size=30, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/46 [00:00<?, ?it/s]

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\github\openai\athina-ai\env\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
  File "c:\github\openai\athina-ai\env\lib\asyncio\base_events.py", line 641, in run_until_complete
    return future.result()
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 83, in _aresults
    raise e
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "c:\github\openai\athina-ai\env\lib\asyncio\tasks.py", line 575, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 37, in sema_coro
    return await coro
  File "c:\github\openai\athina-ai\env\lib\site-packages\ra

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [17]:
testset.to_pandas()

NameError: name 'testset' is not defined

In [18]:
from langchain.vectorstores import pinecone
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_pinecone import PineconeVectorStore

In [19]:
index_name = "policybot"
# Connect to Pinecone index 
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

In [20]:
# genration part
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192"
)

In [21]:
prompt_template="""
You are an a professional AI assistant for car insurance inquiries, \
use the provided information to accurately address the user's question. \
If the answer is not available, clearly state that you do not have the information instead of providing a speculative response.
-----------------------------
Context: {context}
-----------------------------
Question: {question}

Please provide in professional and helpful answer below , without any additional commentary.
Helpful answer:
"""

In [22]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])


In [23]:

parser = StrOutputParser()


In [24]:
compressor = CohereRerank(cohere_api_key="nbDqU1hTVxWmXGbLYI6OnYhp4Cx40MZ5hOmO5oKX")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vectorstore.as_retriever(search_kwargs={"k": 5})
    )


In [25]:
chain = (
    {
        "context":  compression_retriever,
        "question":  RunnablePassthrough(),
    }
    | PROMPT
    | llm
    | parser
)

In [26]:
import pandas as pd
df = pd.read_csv("qa_pairs.csv")

In [28]:
df.rename(columns={"answer": "ground_truth"}, inplace=True)

In [59]:
df = df.sample(n=5).reset_index(drop=True)

In [60]:
df

,question,ground_truth
0,What type of policy is Comprehensive with Driv...,Same as a Comprehensive policy
1,What is the deductible for collision coverage?,$500.
2,What is Excess in the context of car insurance?,The amount that you may have to pay towards a ...
3,Am I covered if I leave my car unlocked or the...,"No, we won’t pay a claim."
4,Where can I find the exact meanings of specifi...,Glossary on page 4 or at the start of each sec...


In [62]:
questions = df["question"].to_list()
ground_truth = df["ground_truth"].to_list()

In [63]:
questions

['What type of policy is Comprehensive with DriveSure?',
 'What is the deductible for collision coverage?',
 'What is Excess in the context of car insurance?',
 'Am I covered if I leave my car unlocked or the keys in the car?',
 'Where can I find the exact meanings of specific words and phrases?']

In [64]:
from datasets import Dataset


data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

for query in questions:
    data["question"].append(query)
    data["answer"].append(chain.invoke(query))
    print(query)
    data["contexts"].append([doc.page_content for doc in compression_retriever.invoke(query)])
    print("incerted")

dataset = Dataset.from_dict(data)

What type of policy is Comprehensive with DriveSure?
incerted
What is the deductible for collision coverage?
incerted
What is Excess in the context of car insurance?
incerted
Am I covered if I leave my car unlocked or the keys in the car?
incerted
Where can I find the exact meanings of specific words and phrases?
incerted


In [98]:
dataset

EvaluationDataset(test_cases=[], goldens=[], conversational_goldens=[], _alias=None, _id=None)

In [66]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

Exception in thread Thread-35:
Traceback (most recent call last):
  File "c:\github\openai\athina-ai\env\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 95, in run
    results = self.loop.run_until_complete(self._aresults())
  File "c:\github\openai\athina-ai\env\lib\asyncio\base_events.py", line 641, in run_until_complete
    return future.result()
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 83, in _aresults
    raise e
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 78, in _aresults
    r = await future
  File "c:\github\openai\athina-ai\env\lib\asyncio\tasks.py", line 575, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\github\openai\athina-ai\env\lib\site-packages\ragas\executor.py", line 37, in sema_coro
    return await coro
  File "c:\github\openai\athina-ai\env\lib\site-packages\r

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [ ]:
result.to_pandas()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

df = result.to_pandas()

heatmap_data = df[['context_relevancy', 'context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']]

cmap = LinearSegmentedColormap.from_list('green_red', ['red', 'green'])

plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".2f", linewidths=.5, cmap=cmap)

plt.yticks(ticks=range(len(df['question'])), labels=df['question'], rotation=0)

plt.show()

In [90]:
prompt_template = """
Your task is to write a factoid question and an answer given a context.
Your factoid question should be answerable with a specific, concise piece of factual information from the context.
Your factoid question should be formulated in the same style as questions users could ask in a search engine.
This means that your factoid question MUST NOT mention something like "according to the passage" or "context".

Provide your answer as follows:

------------
{context}
------------

Create 5 Question and Answer pairs in JSON format.
Make sure not to lose any important information.
here is an example:
\"question\": \"What is AI?\", \"answer\": \"AI stands for Artificial Intelligence.\", \
\"question\": \"What is ML?\", \"answer\": \"ML stands for Machine Learning.\", \
"""

In [91]:
PROMPT_QUESTIONS = PromptTemplate(template=prompt_template, input_variables=["context"])

In [92]:
# genration part
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192"
)

In [93]:
from langchain_core.prompts import PromptTemplate

In [94]:
from langchain_core.output_parsers import BaseOutputParser
from typing import List, Dict
import re
import json

class dictOutput(BaseOutputParser):
    def parse(self, text: str) -> List[Dict[str, str]]:
        # Finding all JSON objects in the text
        matches = re.findall(r'\{[^}]+\}', text)
        
        if not matches:
            print("No JSON objects found in the input text.")
            return []
        
        result = []
        
        for json_text in matches:
            try:
                # Load the JSON data
                data = json.loads(json_text)
                
                # Extract the question and answer
                question = data.get("question")
                answer = data.get("answer")
                
                if question is not None and answer is not None:
                    result.append({
                        "question": question,
                        "answer": answer
                    })
                else:
                    print(f"Invalid JSON object found: missing 'question' or 'answer' key in {json_text}")
            except json.JSONDecodeError:
                print(f"Invalid JSON format found in text: {json_text}")
        
        return result


In [95]:
# Chain for generating question and answer
chain_for_qa = PROMPT_QUESTIONS | llm | dictOutput()


In [97]:
    print(chain_for_qa.invoke(docs[2].page_content))

[{'question': 'What are accessories in car insurance?', 'answer': 'Parts or products specifically designed to be fitted to your car, including electric car’s charging cables and the charger installed at your home.'}, {'question': 'What is an approved repairer in car insurance?', 'answer': 'A repairer in our network of contracted repairers who’s approved by us to carry out repairs to your car following a claim under this policy.'}, {'question': 'What is an automated car in the context of car insurance?', 'answer': 'A car that is lawfully driving itself on roads or other public places in Great Britain.'}, {'question': 'What is a certificate of motor insurance?', 'answer': 'A document that provides evidence that you have taken out the insurance you must have by law.'}, {'question': 'What is excess in car insurance?', 'answer': 'The amount that you may have to pay towards a claim.'}]


In [ ]:
import csv
import time

# Assuming `docs` is a list of documents and each document has a `page_content` attribute
# Also assuming `chain_for_qa` is an object with an `invoke` method that processes the content

# List to hold all question-answer pairs
qa_pairs_list = []

# Maximum number of requests per minute
max_requests_per_minute = 13
# Time interval between requests in seconds
interval = 60 / max_requests_per_minute

# Loop through each document and process its content
for index, doc in enumerate(docs):
    # Get the page content
    page_content = doc.page_content
    
    # Invoke the chain to get question-answer pairs
    qa_pairs = chain_for_qa.invoke(page_content)
    
    # Assuming the output of `invoke` is a list of question-answer pairs
    qa_pairs_list.extend(qa_pairs)
    
    # Wait to respect the rate limit, except after the last request
    if (index + 1) % max_requests_per_minute == 0:
        print("Reached the limit of 13 requests per minute. Waiting for the next minute...")
        time.sleep(60)  # Wait for a minute before making more requests
    else:
        time.sleep(interval)  # Wait for the calculated interval before the next request

# Define the CSV file name
csv_file_name = 'qa_pairs.csv'

# Save the collected question-answer pairs to a CSV file
with open(csv_file_name, mode='w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['question', 'answer']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for qa in qa_pairs_list:
        writer.writerow(qa)

print(f"Question-answer pairs have been saved to {csv_file_name}")


Reached the limit of 13 requests per minute. Waiting for the next minute...
Reached the limit of 13 requests per minute. Waiting for the next minute...
Reached the limit of 13 requests per minute. Waiting for the next minute...
Question-answer pairs have been saved to qa_pairs.csv


In [21]:
import pandas as pd
df = pd.read_csv("qa_pairs.csv")

In [22]:
df.head()

,question,answer
0,What is the policy period?,12 months from the effective date.
1,What is the deductible for collision coverage?,$500.
2,What is the limit for personal liability?,"$100,000."
3,Is roadside assistance included?,Yes.
4,What is the process for filing a claim?,Call 1-800-CLAIMS or report online within 24 h...


In [49]:
df

,question,ground_truth
0,What happens if I choose to repair my car at a...,Those repairs will NOT be guaranteed by the in...
1,Where can I find the exact meanings of specifi...,Glossary on page 4 or at the start of each sec...
2,What is a Certificate of motor insurance?,A document that provides evidence that you hav...
3,What makes up the policy?,"This booklet, car insurance details, certifica..."
4,What if I arrange windscreen repairs or replac...,The insurer will only cover a limited amount.
5,What is a Courtesy car?,"A small hatchback car, or similar car, that an..."
6,What is the limit for replacing a child car seat?,Unlimited
7,What is the excess for accidental damage claim...,£250
8,In what situations do I not need to pay an exc...,If you're in an accident that's not your fault...
9,Who is the 'Policyholder'?,The person named as the policyholder on the ca...


deepeval

In [50]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 50
})

TypeError: EvaluationDataset.__init__() got an unexpected keyword argument 'model'

NameError: name 'test_case' is not defined

In [73]:
data = pd.DataFrame(dataset)

In [75]:
data.head()

EvaluationDataset(test_cases=[], goldens=[], conversational_goldens=[], _alias=None, _id=None)

In [ ]:
from haystack import Pipeline
from haystack_integrations.components.evaluators.deepeval import DeepEvalEvaluator, DeepEvalMetric

QUESTIONS = df["question"]
CONTEXTS = [
    [
        "The popularity of sports can be measured in various ways, including TV viewership, social media presence, number of participants, and economic impact.",
        "Football is undoubtedly the world's most popular sport with major events like the FIFA World Cup and sports personalities like Ronaldo and Messi, drawing a followership of more than 4 billion people.",
    ],
    [
        "Python, created by Guido van Rossum in the late 1980s, is a high-level general-purpose programming language.",
        "Its design philosophy emphasizes code readability, and its language constructs aim to help programmers write clear, logical code for both small and large-scale software projects.",
    ],
]
RESPONSES = [
    "Football is the most popular sport with around 4 billion followers worldwide",
    "Python language was created by Guido van Rossum.",
]

pipeline = Pipeline()
evaluator = DeepEvalEvaluator(
    metric=DeepEvalMetric.FAITHFULNESS,
    metric_params={"model": "gpt-4"},
)
pipeline.add_component("evaluator", evaluator)

# Each metric expects a specific set of parameters as input. Refer to the
# DeepEvalMetric class' documentation for more details.
results = pipeline.run({"evaluator": {"questions": QUESTIONS, "contexts": CONTEXTS, "responses": RESPONSES}})

for output in results["evaluator"]["results"]:
    print(output)
